# Notebook 4 - Education and Salary - a first overview.

This notebook aims to create a single dataframe collecting data from the _Census.Gov_ page with information on:
1. Education Levels - Namely the percentage of Over25 that hold a Bachelor's degree
2. Education Institution Presence - from the Firms API we analysed the "educational services" category (number 61) that showed us per each county how many educ. centres were present (this includes kindergarten, schools of any level and university)
3. Salary - The total income of the county

**PLEASE NOTE** While the aim of our research is to show an uniform patter in all states of the United States, we will only take into consideration the state of California, in order to speed up the output results.


We start by importing the necessary libraries, retrieving the data from the API and analysing only the state of California for the sake of notebook size, we will proceed to a further analysis in a later step of our project

In [2]:
#Required imports for the project
import requests # for api requests
import pandas as pd #tabular data

from bs4 import BeautifulSoup

In [3]:
# !! REMINDER TO TAKE API KEY OUT OF CODE BEFORE SUBMITTING !!
api_key = ""

In [4]:
#creating a dataframe function
def json_to_dataframe(response):
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [41]:
#Creating a function to remove the leading zeros from a column
def remove_leading_zeros(df, column):
    df[column] = df[column].str.lstrip('0')
    return df

In [5]:
#Setting up the API Query parameters for the educational services industry
params1 = {"NAICS2017" : 61}

In [6]:
#Requesting the json file from the census website using the api key
url = "https://api.census.gov/data/2017/ecnbasic?get=NAICS2017_LABEL,NAICS2017,GEO_ID,FIRM&for=county:*&key={}".format(api_key)
response = requests.request("GET", url, params=params1)

In [7]:
response.text[0:77]

'[["NAICS2017_LABEL","NAICS2017","GEO_ID","FIRM","NAICS2017","state","county"]'

In [8]:
#Converting ecn data frame into pandas data frame
educational_services = json_to_dataframe(response)
df = pd.DataFrame(data = educational_services)
df['FIRM'] = df['FIRM'].astype(int)
df['state'] = df['state'].astype("string")
df_california1 = df[df['state'] == '06']

In [9]:
# Top 5 counties with largest number of educational services
df_c_sort = df_california1\
.sort_values(by=['FIRM'], ascending=False)\
.head(10)
print('Top 10 counties with the most amount of educational services in California')
df_c_sort

Top 10 counties with the most amount of educational services in California


,NAICS2017_LABEL,NAICS2017,GEO_ID,FIRM,NAICS2017,state,county
288,Educational services,61,0500000US06037,2787,61,06,037
304,Educational services,61,0500000US06059,1187,61,06,059
319,Educational services,61,0500000US06073,1010,61,06,073
291,Educational services,61,0500000US06085,822,61,06,085
201,Educational services,61,0500000US06001,693,61,06,001
206,Educational services,61,0500000US06075,470,61,06,075
202,Educational services,61,0500000US06067,334,61,06,067
308,Educational services,61,0500000US06081,323,61,06,081
289,Educational services,61,0500000US06065,313,61,06,065
196,Educational services,61,0500000US06013,295,61,06,013


In [10]:
#Setting the params
params2 = {"state" : "06"}

In [25]:
#Getting the ACS data
#Requesting the json file from the census website using the api key 
url = "https://api.census.gov/data/2017/acs/acs1/profile?get=DP02_0064PE,DP02_0088PE,DP02_0123PE&for=county&key={}".format(api_key)
response2 = requests.request("GET", url)

In [26]:
#Seeing what the columns for the data are
response2.text[0:61]

'[["DP02_0064PE","DP02_0088PE","DP02_0123PE","state","county"]'

## Meaning of variables
- **DP02_0064PE** = Percent!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree

In [13]:
pop_chars = json_to_dataframe(response2)
df = pd.DataFrame(data = pop_chars)
df_california2 = df[df['state'] == '06']

In [77]:
#Merging the two datasets
merged_census = pd.merge(df_california1, df_california2, on='county')
#drop state_y column and rename state_x to state
merged_census = merged_census.drop(columns=['state_y', 'NAICS2017'])
merged_census = merged_census.rename(columns={'state_x': 'state', 'FIRM':'Number of Educational Institutions', 'DP02_0064PE':'Percent of Population with a Bachelor\'s Degree'})
#Convert the columns to the correct data types
merged_census['Percent of Population with a Bachelor\'s Degree'] = merged_census['Percent of Population with a Bachelor\'s Degree'].astype(float)
merged_census['county'] = merged_census['county'].astype(int)
# The last two columns are currently unneeded but will be later used for contextual analysis.
#Drop the last two columns
merged_census = merged_census.drop(columns=['DP02_0088PE', 'DP02_0123PE'])
#Remove leading zeros from county column
##merged_census = remove_leading_zeros(merged_census, 'county')
merged_census

,NAICS2017_LABEL,GEO_ID,Number of Educational Institutions,state,county,Percent of Population with a Bachelor's Degree
0,Educational services,0500000US06047,13,06,47,8.7
1,Educational services,0500000US06033,8,06,33,9.5
2,Educational services,0500000US06115,0,06,115,13.9
3,Educational services,0500000US06013,295,06,13,26.7
4,Educational services,0500000US06083,137,06,83,21.2
5,Educational services,0500000US06097,161,06,97,22.6
6,Educational services,0500000US06025,5,06,25,10.4
7,Educational services,0500000US06001,693,06,1,26.9
8,Educational services,0500000US06067,334,06,67,20.0
9,Educational services,0500000US06017,37,06,17,22.7


In [84]:
#Getting the list of counties in the ACS data for california
acs_counties = merged_census['county'].tolist()
acs_counties
#Getting the list of county names from the census website
ca_counties = pd.read_csv("file:/home/leomassoc/Workspace/LSE-DS105L-Data-Detectives/data/cal_counties.csv")
#Drop the state name and state number columns
ca_counties = ca_counties.drop(columns=['State_Name', 'State'])
ca_counties

,County,County_Name
0,1,Alameda County
1,3,Alpine County
2,5,Amador County
3,7,Butte County
4,9,Calaveras County
5,11,Colusa County
6,13,Contra Costa County
7,15,Del Norte County
8,17,El Dorado County
9,19,Fresno County


In [88]:
#Merg the merged_census dataframe with the ca_counties dataframe to get the county names
merged_census = pd.merge(merged_census, ca_counties, left_on= 'county', right_on = 'County')
merged_census = merged_census.drop(columns=['County_x', 'County_y', 'County_Name_x'])
merged_census

,NAICS2017_LABEL,GEO_ID,Number of Educational Institutions,state,county,Percent of Population with a Bachelor's Degree,County_Name_y
0,Educational services,0500000US06047,13,06,47,8.7,Merced County
1,Educational services,0500000US06033,8,06,33,9.5,Lake County
2,Educational services,0500000US06115,0,06,115,13.9,Yuba County
3,Educational services,0500000US06013,295,06,13,26.7,Contra Costa County
4,Educational services,0500000US06083,137,06,83,21.2,Santa Barbara County
5,Educational services,0500000US06097,161,06,97,22.6,Sonoma County
6,Educational services,0500000US06025,5,06,25,10.4,Imperial County
7,Educational services,0500000US06001,693,06,1,26.9,Alameda County
8,Educational services,0500000US06067,334,06,67,20.0,Sacramento County
9,Educational services,0500000US06017,37,06,17,22.7,El Dorado County


## Getting the Code on National Income

In [100]:
#importing the excel file called "GDP_County_2021_NotPerCapita.xlsx" and transforming it into a pandas dataframe
###change the directory to the location of the excel file on your computer

gdp = pd.read_excel("file:GDP_County_2021_NotPerCapita.xlsx")
gdp = dpg[]
#gdp[StateName == "California"]

Table 1


,StateName,CountyName,GDP
0,Alabama,Autauga,1502153
1,Alabama,Baldwin,7830237
2,Alabama,Barbour,709459
3,Alabama,Bibb,392249
4,Alabama,Blount,997835
...,...,...,...
3115,Wyoming,Sweetwater,3125840
3116,Wyoming,Teton,2862327
3117,Wyoming,Uinta,771851
3118,Wyoming,Washakie,346859


In [ ]:
codes = pd.read_excel("file:/Users/albertomartino/Desktop/LSE/DS105/Summatives/Project/LSE-DS105L-Data-Detectives/notebooks/fips2county.xlsx")
codes.head(20)
#In CountyFIPS column, if the value is under 10000, add a 0 in front of it
codes['CountyFIPS'] = codes['CountyFIPS'].apply(lambda x: '{0:0>5}'.format(x))
codes.head(20) 

,StateFIPS,CountyFIPS_3,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY
0,1,1,Autauga,Alabama,01001,AL,AL | AUTAUGA
1,1,3,Baldwin,Alabama,01003,AL,AL | BALDWIN
2,1,5,Barbour,Alabama,01005,AL,AL | BARBOUR
3,1,7,Bibb,Alabama,01007,AL,AL | BIBB
4,1,9,Blount,Alabama,01009,AL,AL | BLOUNT
5,1,11,Bullock,Alabama,01011,AL,AL | BULLOCK
6,1,13,Butler,Alabama,01013,AL,AL | BUTLER
7,1,15,Calhoun,Alabama,01015,AL,AL | CALHOUN
8,1,17,Chambers,Alabama,01017,AL,AL | CHAMBERS
9,1,19,Cherokee,Alabama,01019,AL,AL | CHEROKEE


In [ ]:
#For each row of GDP get the CountyFIPS corresponding to the county name and state in the codes dataframe
#Then add the CountyFIPS to the GDP dataframe

### TO - DO

# 🚧 Work in Progress - Data Visualisation: Is there a relation between GDP and Education Level? 🚧
@Seyi and @Alua will complete this section soon.